In [ ]:
# import math
import pandas as pd
import numpy as np
import requests
from io import StringIO

import yfinance as yf
%matplotlib inline
import matplotlib.pyplot as plt

1. Liquidity: currency of the bear market
    1. market impact: stay mid-large caps on the short side
    2. Market Cap: function and fields
2. Exchange: OTC
4. Crowded shorts: 
    1. Percentage of the free float
    2. ratios and prudence

4. Valuations
    1. P/E : poor predictor of imminent shorts. Value traps are often cheap
    2. High PBR indicate highly leveraged balance sheet. Everything is about timing
    3. Dividend yield:
        1. avg 5 year and underperformance
        2. navigate ex-date


### Downloading information for all S&P 500 companies

How it works:
1. Fetch data: Request Wikipedia page with browser headers to avoid blocking
2. Extract table: Parse HTML and select the second table [1] containing company data
3. Clean up: Rename 'GICS Sector' → 'sector' and set ticker symbols as index
4. Verify: Display first 5 rows
5. Result: DataFrame indexed by ticker (e.g., AAPL, MSFT) with columns for company name, sector, date added, etc.

In [ ]:

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}  # (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response= requests.get(url, headers=headers)
df_SP500 = pd.read_html(StringIO(response.text))[0]
df_SP500 = df_SP500.rename(columns={'GICS Sector':'sector'})
df_SP500.set_index('Symbol', inplace=True)

bm_ticker = '^GSPC'
tickers = [bm_ticker] + df_SP500.index.tolist() 
df_SP500.head()

In [ ]:
def batch_px_df(tickers_list,batch_size, start, end,show_batch = False):
    px_df = pd.DataFrame()
    loop_size = int(len(tickers_list) // batch_size) + 2
    for t in range(1,loop_size): # Batch download
        try:
            m = (t - 1) * batch_size
            n = t * batch_size
            batch_list = tickers_list[m:n]
            if show_batch:
                print(batch_list,m,n)
            batch_download = yf.download(tickers= batch_list,start= start, end = end, interval = "1d",
                                group_by = 'column',auto_adjust = True, prepost = True)['Close']
            px_df = px_df.join(batch_download, how='outer')
        except:
            pass
    return px_df.tz_localize(None)

### Fetch detailed company information for all S&P 500 tickers from YFinance

How it works:
1. Loop through tickers: Iterate over each S&P 500 symbol
2. Fetch data: Use yf.Ticker(symbol).info to get company fundamentals (market cap, PE ratio, dividends, short interest, etc.)
3. Error handling: If fetch fails, store None and print error message
4. Build DataFrame: Convert dictionary to DataFrame with tickers as index

Result: DataFrame with ~500 rows (one per company) and 100+ columns containing financial metrics, ratios, and company metadata.

In [ ]:
sp500_info_dict = {}

for symbol in tickers:
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info
        sp500_info_dict[symbol] = info
    except Exception as e:
        print(f"Error fetching info for {symbol}: {e}")
        sp500_info_dict[symbol] = None
        
sp500_info_df= pd.DataFrame.from_dict(sp500_info_dict, orient='index')
# sp500_info_df.to_csv('sp500_info.csv')

### Liquidity is the currency of bear markets

Clean and engineer key screening metrics for short selling: liquidity, market cap, and short interest indicators.

How it works:
1. Date formatting: Convert dividend dates from Unix timestamps to YYYY-MM-DD format
2. Market cap fix: Calculate market cap from shares × price when API value is zero; convert to millions
3. Liquidity metric: Compute daily dollar volume traded (prefer 10-day average over standard average)
4. Short interest: Calculate short % of float from raw shares data; fill missing values from API field
5. Display subset: Show key columns for liquidity, shorts, valuations, and dividends

Result: Cleaned DataFrame with standardized metrics ready for screening high-liquidity, heavily-shorted, or dividend-paying stocks.

In [ ]:
sp500_info_df[['exDividendDate', 'dividendDate']] = sp500_info_df[['exDividendDate', 'dividendDate']].apply(lambda x: pd.to_datetime(x).dt.strftime('%Y-%m-%d'))

# Market Cap correction
sp500_info_df['marketCap_calculated'] = sp500_info_df['sharesOutstanding'].mul(sp500_info_df['currentPrice'])
sp500_info_df['market_Cap']  = np.where(sp500_info_df['marketCap'] == 0, sp500_info_df['marketCap_calculated'], sp500_info_df['marketCap'])
sp500_info_df['market_Cap']  = round(sp500_info_df['market_Cap'] *1e-6)

# Average Liquidity calculation
value_traded  = round(sp500_info_df['averageVolume'].mul(sp500_info_df['currentPrice']) *1e-6)
value_traded_avg10  = round(sp500_info_df['averageDailyVolume10Day'].mul(sp500_info_df['currentPrice']) *1e-6)
sp500_info_df['value_traded'] = np.where(~pd.isna(value_traded_avg10), value_traded_avg10, value_traded)

# Crowded shorts
sp500_info_df['shortfloat'] = round(sp500_info_df['sharesShort'].div(sp500_info_df['floatShares']),3)
sp500_info_df['shortPctFloat'] = np.where(~pd.isna(sp500_info_df['shortfloat']), 
                                          sp500_info_df['shortfloat'], sp500_info_df['shortPercentOfFloat'])

sp500_info_df[['shortName','sector','exchange','market_Cap', 'value_traded', 'currentPrice',#'sharesOutstanding', 'averageDailyVolume10Day', 'averageVolume',
 'shortPctFloat', 'shortRatio', #'sharesShort', 'floatShares', 'shortPercentOfFloat',
'exDividendDate', 'dividendDate','dividendYield', 'fiveYearAvgDividendYield', 'payoutRatio',
'trailingPE', 'forwardPE', 'priceToBook', ]].head(10)

### Identify the most heavily shorted stocks by short ratio

Screen for stocks with the highest short ratio (days to cover) to find crowded short positions that may face squeeze risk.

How it works:
1. Rank stocks: Calculate percentile rank of shortRatio across all S&P 500 stocks
2. Sort and select: Sort descending and take top 10 tickers with highest ratios
3. Display metrics: Show shortPctFloat and shortRatio for these heavily-shorted names
4. Interpretation: High short ratio means many days of average volume needed to cover all short positions

Result: Top 10 stocks where shorts may face difficulty exiting positions quickly, indicating potential squeeze candidates or genuine distress.

In [ ]:
short_ratio = sp500_info_df['shortRatio'].rank(pct=True).sort_values(ascending=False).index[:10].tolist()
sp500_info_df.loc[short_ratio, ['shortName','sector','shortPctFloat', 'shortRatio']].round(3).head(10)

In [ ]:
sp500_info_df.groupby('sector')[['shortPctFloat','shortRatio']].mean().sort_values(by='shortPctFloat', ascending=False).round(3)

### Identify the most heavily shorted stocks by short as a percentage of free float

Screen for stocks with the highest short as a percentage of free float to find crowded short positions that may face squeeze risk.

How it works:
1. Rank stocks: Calculate percentile rank of shortRatio across all S&P 500 stocks
2. Sort and select: Sort descending and take top 10 tickers with highest ratios
3. Display metrics: Show shortPctFloat and shortRatio for these heavily-shorted names
4. Interpretation: High short as a percentage of free float means demand for short (shares borrowed) has increased as a percentage of the supply (free float)

In [ ]:
high_short_float = sp500_info_df['shortPercentOfFloat'].rank(pct=True).sort_values(ascending=False).index[:10].tolist()
sp500_info_df.loc[high_short_float, ['shortName','sector','shortPercentOfFloat', 'shortRatio']].round(3).head(10)

### Analyze dividend metrics by sector

Compare dividend characteristics across sectors to identify high-yield sectors and assess dividend sustainability.

How it works:
1. Group by sector: Aggregate all S&P 500 stocks by GICS sector
2. Calculate means: Compute average dividendYield, fiveYearAvgDividendYield, and payoutRatio per sector
3. Sort by 5-year average: Rank sectors from highest to lowest historical dividend yield

Interpretation: In chapter 3 we calculated returns by group. We identified defensives and cyclical sectors. Defensive sectors generally underperform the benchmark. They therefore have to compensate their shareholders with generous dividend yields. 

In [ ]:
sp500_info_df.groupby('sector')[['dividendYield', 'fiveYearAvgDividendYield', 'payoutRatio']].mean().sort_values(by='fiveYearAvgDividendYield', ascending=False).round(2)

In [ ]:
sp500_info_df[['exDividendDate', 'dividendDate']].tail()

### Compare valuation multiples across sectors

Examine price-to-earnings and price-to-book ratios by sector to identify relatively expensive or cheap sectors.

How it works:
1. Group by sector: Aggregate stocks by GICS sector classification
2. Calculate medians: Use median (not mean) for trailingPE, forwardPE, and priceToBook to avoid outlier distortion
3. Sort by trailing PE: Rank sectors from most to least expensive based on historical earnings

Interpretation: Technology have sported gravity defying P/E and still lead the charge in the current bull market. Forward looking P/E look reasonable while trailing P/E are rich. This means earnings momentum and growth prospects are strong. The moment earnings momentum slows down, valuations will compress and those stocks will have a rude encounter with Newtonian physics.
High PBR indicates high leveraged balance sheets compared to valuations. As soon as the rent of money goes up, fragile sheets crumble.

In [ ]:
sp500_info_df.groupby('sector')[['trailingPE', 'forwardPE', 'priceToBook']].median().sort_values(by='trailingPE', ascending=False).round(1)

In [ ]:
batch_size = 25 ; start= '2015-01-01' ; end = None

px_df = batch_px_df(tickers,batch_size, start, end,show_batch = False)
px_df.columns.names = ['ticker']
px_df = px_df.round(2) 
px_df_rel = round(px_df.divide(px_df[bm_ticker], axis=0).mul(px_df.iloc[0,list(px_df.columns).index(bm_ticker)]),1)

px_df_rel = px_df_rel.round(2) 

### Download historical price data and calculate relative performance

Batch download daily close prices for all S&P 500 stocks and compute relative performance vs. benchmark.

How it works:
1. Set parameters: Define batch size (25 tickers/batch), start date (2015-01-01), and benchmark (^GSPC)
2. Batch download: Use batch_px_df() to download all tickers in chunks to avoid API rate limits
3. Clean data: Round prices to 2 decimals and remove timezone info
4. Calculate relative prices: Divide each stock by benchmark and normalize to benchmark's starting value
5. Result: Two DataFrames - px_df (absolute prices) and px_df_rel (performance relative to S&P 500)

Result: Time series of daily prices where px_df_rel shows which stocks outperformed (>100) or underperformed (<100) vs. the benchmark since 2015.

In [ ]:
def beta(df,bm_col):
    log_returns = np.log(df/df.shift(1))
    var = log_returns[bm_col].var()
    cov = log_returns.cov()
    beta = round(cov.loc[:,bm_col] / var,2)
    return beta

### Define beta calculation function

Calculate stock beta (systematic risk) relative to benchmark using log returns covariance method.

How it works:
1. Compute log returns: Calculate log(price_t / price_t-1) for all stocks and benchmark
2. Calculate variance: Compute variance of benchmark returns
3. Compute covariance: Build covariance matrix between all stocks and benchmark
4. Calculate beta: Divide each stock's covariance with benchmark by benchmark variance
5. Round results: Return betas rounded to 2 decimals

Result: Beta values where β>1 indicates higher volatility than market, β<1 means lower volatility, and β≈1 moves with market.

In [ ]:
month_end_list = px_df.resample('ME', closed='left', label='right').last().index.to_list()

window = 12 * 3
beta_df = pd.DataFrame( )
bm_col = '^GSPC'
for t,dt in enumerate(month_end_list):
    beta_monthly_df =  beta(px_df[dt:month_end_list[t + window]],bm_col)
    beta_df = pd.concat([beta_df,beta_monthly_df],axis =1)
    beta_df = beta_df.rename(columns={bm_col: month_end_list[t + window]})
    if month_end_list[t + window] == month_end_list[-1]:
        print(t,dt,month_end_list[t+window])
        break

beta_df.tail()

### Calculate rolling 36-month beta for all stocks

Compute monthly updated betas using a 3-year rolling window to track how systematic risk evolves over time.

How it works:
1. Create month-end list: Resample price data to month-end dates for period boundaries
2. Set rolling window: Use 36 months (12 × 3) for beta stability while capturing regime changes
3. Loop through months: For each month, calculate beta using data from that month through +36 months ahead
4. Build beta DataFrame: Each column represents betas as of a specific month-end date
5. Stop at data end: Break loop when reaching the last available month

Result: DataFrame with betas for each stock (rows) calculated at each month-end (columns), showing how risk profiles change across market cycles.

In [ ]:
df_SP500_beta = pd.DataFrame()
df_SP500_beta = pd.concat([sp500_info_df['sector'], beta_df], axis=1)

sector_beta_pivot = pd.pivot_table(df_SP500_beta, values=list(beta_df.columns), 
                                   index= ['sector'], aggfunc="mean").T
sector_beta_pivot.plot(figsize=(15,4),title = 'Beta by Sector', grid=True)
plt.legend(loc='lower left')

### Visualize beta trends by sector over time

Aggregate individual stock betas to sector level and plot time series to identify which sectors become more/less risky.

How it works:
1. Merge data: Combine sector labels from sp500_info_df with beta time series
2. Pivot table: Calculate mean beta per sector for each month using pd.pivot_table()
3. Transpose: Flip table so months are rows (x-axis) and sectors are columns (series)
4. Plot: Generate time series chart showing beta evolution by sector

Result: Line chart revealing which sectors (e.g., Technology, Energy) exhibit higher systematic risk and how sector betas shift during bull/bear markets.

In [ ]:
daily_log_returns = np.log(px_df/px_df.shift(1))
daily_rel_log_returns = daily_log_returns.sub(daily_log_returns[bm_col],axis=0)
cum_returns_abs = daily_log_returns.cumsum().apply(np.exp)-1
cum_returns_rel = daily_rel_log_returns.cumsum().apply(np.exp)-1

### Calculate absolute and relative returns for strategy analysis

Compute daily log returns in both absolute and benchmark-relative terms for performance attribution.

How it works:
1. Daily log returns: Calculate log(price_t / price_t-1) for all stocks
2. Relative log returns: Subtract benchmark returns from each stock's returns (excess returns)
3. Cumulative absolute: Apply exp(cumsum(log_returns)) - 1 to get total returns
4. Cumulative relative: Apply exp(cumsum(relative_log_returns)) - 1 to get benchmark-adjusted returns

Result: Four DataFrames enabling analysis of both absolute performance and alpha generation (outperformance vs. S&P 500).

### Calculate and visualize sector performance metrics

**Objective:** Compare sector-level absolute and relative returns to identify which sectors outperform or underperform the S&P 500 benchmark.

**Steps:**
1. Merge sector labels with cumulative return time series to organize returns by sector classification
2. Group returns by sector and calculate mean performance across all constituent stocks within each sector
3. Plot both absolute returns (total gains) and relative returns (excess over benchmark) for sector comparison

**Result:** Two visualizations showing sector performance trends—absolute chart reveals total returns while relative chart highlights which sectors generated alpha (outperformance) versus the benchmark.

In [ ]:
sector_abs_returns = pd.concat([sp500_info_df['sector'],cum_returns_abs.T],axis=1)
sector_avg_abs_returns = sector_abs_returns.groupby('sector').mean().T
sector_avg_abs_returns.plot(figsize=(15,4), title='Sector Average Absolute Returns vs S&P 500', grid=True)
plt.legend(loc='upper left')
plt.show()

sector_rel_returns = pd.concat([sp500_info_df['sector'],cum_returns_rel.T],axis=1)
sector_avg_rel_returns = sector_rel_returns.groupby('sector').mean().T
sector_avg_rel_returns.plot(figsize=(15,4), title='Sector Average Relative Returns vs S&P 500', grid=True)
plt.legend(loc='upper left')
plt.show()


### Compare high-beta vs. low-beta sector performance

Analyze performance difference between cyclical high-beta sectors (Technology, Consumer Cyclical) and defensive low-beta sectors (Utilities, Consumer Defensive).

How it works:
1. Define sector groups: Select high-beta sectors (Technology, Consumer Cyclical) and low-beta sectors (Utilities, Consumer Defensive)
2. Filter returns: Extract returns for stocks in each sector group from sector_rel_returns and sector_abs_returns
3. Calculate averages: Compute mean returns across all stocks in high-beta and low-beta groups
4. Build comparison DataFrame: Combine benchmark, high/low beta averages, and their spread (high minus low)
5. Plot relative returns: Chart showing relative performance vs. S&P 500 for both groups and their difference
6. Plot absolute returns: Chart showing total returns including benchmark for comparison

Result: Two charts demonstrating beta factor premium—high-beta sectors outperform in bull markets but underperform in bear markets, while low-beta sectors provide defensive characteristics. The spread line shows the cyclical nature of the beta factor.

Note: The absolute chart shows the returns without adjustment for the benchmark returns. This is why it looks so much better. The relative chart shows the excess returns over the benchmark.

In [ ]:
high_beta_sectors = ['Consumer Cyclical', 'Technology']
low_beta_sectors = ['Consumer Defensive', 'Utilities']

high_beta_rel_returns = sector_rel_returns.loc[sector_rel_returns['sector'].isin(high_beta_sectors)].drop(columns=['sector'])
low_beta_rel_returns = sector_rel_returns.loc[sector_rel_returns['sector'].isin(low_beta_sectors)].drop(columns=['sector'])

high_beta_abs_returns = sector_abs_returns.loc[sector_abs_returns['sector'].isin(high_beta_sectors)].drop(columns=['sector'])
low_beta_abs_returns = sector_abs_returns.loc[sector_abs_returns['sector'].isin(low_beta_sectors)].drop(columns=['sector'])

beta_cum_returns = pd.DataFrame()
beta_cum_returns[bm_col]= cum_returns_abs[bm_col].copy()
beta_cum_returns['high_beta_rel_avg'] = high_beta_rel_returns.mean(axis=0)
beta_cum_returns['low_beta_rel_avg'] = low_beta_rel_returns.mean(axis=0)
beta_cum_returns['high_low_beta_rel_avg'] = beta_cum_returns['high_beta_rel_avg'].sub(
    beta_cum_returns['low_beta_rel_avg'] )

beta_cum_returns['high_beta_abs_avg'] = high_beta_abs_returns.mean(axis=0)
beta_cum_returns['low_beta_abs_avg'] = low_beta_abs_returns.mean(axis=0)
beta_cum_returns['high_low_beta_abs_avg'] = beta_cum_returns['high_beta_abs_avg'].sub(
    beta_cum_returns['low_beta_abs_avg'] )

beta_cum_returns[[bm_col,'high_beta_rel_avg','low_beta_rel_avg', 'high_low_beta_rel_avg']].plot(
    grid = True, figsize= (15,4), title = 'High Low Beta Sectors Relative Returns')

beta_cum_returns[[bm_col,'high_beta_abs_avg','low_beta_abs_avg', 'high_low_beta_abs_avg']].plot(
    grid = True, figsize= (15,4), title = 'High Low Beta Sectors Absolute Returns')

### Combine sector information with cumulative returns

Merge sector labels with cumulative return matrices to enable sector-level performance analysis.

How it works:
1. Transpose returns: Flip cum_returns DataFrames so tickers are rows (matching sp500_info_df index)
2. Concatenate sector: Add 'sector' column from sp500_info_df to returns data
3. Create both versions: Build sector_rel_returns (benchmark-adjusted) and sector_abs_returns (total returns)

Result: DataFrames indexed by ticker with sector label + all daily cumulative returns, ready for groupby operations to analyze sector performance.

### Backtest high-beta vs. low-beta stock strategy

This code backtests a daily-rebalanced beta factor strategy tracking high-beta vs. low-beta stock portfolios.

Steps:
1. Each month, select the 5 highest-beta and 5 lowest-beta stocks based on previous month's rankings
2. Calculate mean daily relative log returns across the 5 stocks in each portfolio during the holding period
3. Concatenate all daily returns across holding periods into continuous time series
4. Compute the long-short spread (top minus bottom), add benchmark returns, and apply cumulative sum
5. Plot cumulative performance showing how each strategy performs over time vs. the benchmark

In [ ]:
pos_count = 5
top_beta_returns_list = []
bottom_beta_returns_list = []
beta_returns_df = pd.DataFrame()

for t in range(1,len(beta_df.columns)):
    period_start = beta_df.columns[t-1]
    period_end = beta_df.columns[t]

    top_beta = beta_df[period_start].nlargest(n = pos_count).index
    bottom_beta = beta_df[period_start].nsmallest(n = pos_count).index
    top_beta_returns = daily_rel_log_returns.loc[period_start:period_end, top_beta].iloc[1:].mean(axis=1)#.sum()
    bottom_beta_returns = daily_rel_log_returns.loc[period_start:period_end, bottom_beta].iloc[1:].mean(axis=1)#.sum()

    top_beta_returns_list.append(top_beta_returns)
    bottom_beta_returns_list.append(bottom_beta_returns)

top_beta_returns_df = pd.concat(top_beta_returns_list)
bottom_beta_returns_df = pd.concat(bottom_beta_returns_list)

beta_returns_df[f'top_{pos_count}_beta']  = top_beta_returns_df
beta_returns_df[f'bottom_{pos_count}_beta']  = bottom_beta_returns_df
beta_returns_df[f'net_{pos_count}_beta']  = beta_returns_df[f'top_{pos_count}_beta'].sub(beta_returns_df[f'bottom_{pos_count}_beta'])
beta_returns_df[bm_col] = daily_log_returns[bm_col]
beta_returns_df = beta_returns_df.cumsum()
beta_returns_df.plot(figsize=(15,4), grid = True, title=f'Top vs Bottom Beta {pos_count} Stocks Cumulative Returns'  )

### Backtest beta momentum strategy (rising vs. falling betas)

This code tests a strategy based on beta changes rather than absolute beta levels, targeting stocks with rising or falling systematic risk.

Steps:
1. Calculate month-over-month percentage change in beta values for all stocks to identify momentum in risk profiles
2. Each month, select the 5 stocks with the largest beta increases (rising risk) and 5 with the largest beta decreases (falling risk)
3. Calculate mean daily relative log returns for each portfolio during the holding period
4. Concatenate daily returns into continuous time series, compute long-short spread (falling minus rising), and add benchmark
5. Plot cumulative performance to determine if betting against beta momentum (short rising, long falling) generates alpha

In [ ]:
beta_diff = beta_df.pct_change(axis=1, fill_method = None).dropna(axis=1, how = 'all')
top_beta_diff_returns_list = []
bottom_beta_diff_returns_list = []
beta_returns_diff = pd.DataFrame()

for t in range(1,len(beta_diff.columns)):
    period_start = beta_diff.columns[t-1]
    period_end = beta_diff.columns[t]

    top_beta = beta_diff[period_start].nlargest(n = pos_count).index
    bottom_beta = beta_diff[period_start].nsmallest(n = pos_count).index
    top_beta_diff_returns = daily_rel_log_returns.loc[period_start:period_end, top_beta].iloc[1:].mean(axis=1)#.sum()
    bottom_beta_diff_returns = daily_rel_log_returns.loc[period_start:period_end, bottom_beta].iloc[1:].mean(axis=1)#.sum()

    top_beta_diff_returns_list.append(top_beta_diff_returns)
    bottom_beta_diff_returns_list.append(bottom_beta_diff_returns)
top_beta_diff_returns_df = pd.concat(top_beta_diff_returns_list)
bottom_beta_diff_returns_df = pd.concat(bottom_beta_diff_returns_list)

beta_returns_diff[f'rising_{pos_count}_beta']  = top_beta_diff_returns_df
beta_returns_diff[f'falling_{pos_count}_beta']  = bottom_beta_diff_returns_df
beta_returns_diff[f'net_{pos_count}_beta']  = beta_returns_diff[f'falling_{pos_count}_beta'].sub(beta_returns_diff[f'rising_{pos_count}_beta'])
beta_returns_diff[bm_col] = daily_log_returns[bm_col]
beta_returns_diff = beta_returns_diff.cumsum()
beta_returns_diff.plot(figsize=(15,4), grid = True, title=f'{pos_count} Fallling V. Rising Betas Cumulative Returns'  )